In [1]:
import requests
import json
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_utc_timestamp, to_timestamp, to_date, col
import nltk
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot as plt
import datetime
import time
import pickle


import matplotlib.dates as mdates
import matplotlib.pyplot as plt 



Bad key "text.kerning_factor" on line 4 in
/h/224/cameron/miniconda3/envs/pyspark_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
PUSHSHIFT_REDDIT_URL = "http://api.pushshift.io/reddit"

def fetchObjects(**kwargs):
    # Default params values
    params = {"sort_type":"created_utc","sort":"asc","size":1000}
    for key,value in kwargs.items():
        params[key] = value
    print(params)
    type = "comment"
    if 'type' in kwargs and kwargs['type'].lower() == "submission":
        type = "submission"
    r = requests.get(PUSHSHIFT_REDDIT_URL + "/" + type + "/search/",params=params)
    if r.status_code == 200:
        response = json.loads(r.text)
        data = response['data']
        sorted_data_by__id = sorted(data, key=lambda x: int(x['id'],36))
        return sorted_data_by__id

def process(**kwargs):
    max_created_utc = 1577750400
    max_id = 0
    data = pd.DataFrame()    
    #file = open("data.json","w")
    while 1:
        nothing_processed = True
        d = {'created_utc': [], 'body': [], 'subreddit': [], 'score': [],'author':[]}
        objects = fetchObjects(**kwargs,after=max_created_utc)
        if objects == None:
            print("pushshift failed")
            continue
        for object in objects:
            id = int(object['id'],36)
            if id > max_id:
                nothing_processed = False
                created_utc = object['created_utc']
                max_id = id
                if created_utc > max_created_utc: max_created_utc = created_utc
                # Code to do something with comment goes here ...
                # ...
                # insertCommentIntoDB(object)
                # print(json.dumps(object,sort_keys=True,ensure_ascii=True),file=file)
                # ...
                d['body'].append(object['body'])
                d['created_utc'].append(created_utc)
                d['author'].append(object['author'])
                d['subreddit'].append(object['subreddit'])
                d['score'].append(object['score'])
        data = data.append(pd.DataFrame(d))
        if nothing_processed: 
            data.to_parquet("dataframes/2020_comments/{subreddit}-2020_comments.parquet".format(**kwargs))
        max_created_utc -= 1
        time.sleep(1)

subreddits = ["Pete_Buttigieg"]
frames = map(lambda sub : process(subreddit=sub,type="comment",before=1588291200),subreddits)
data = pd.concat(frames)
data

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577750400}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577753296}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577756667}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577760735}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577765384}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577768271}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578009459}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578012230}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578015983}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578020107}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578022831}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578027134}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578276302}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578281823}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578295769}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578307312}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578319966}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578323496}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578538309}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578541253}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578544814}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578551435}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578567270}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578576027}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578746160}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578755885}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578760359}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578763292}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578767707}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578770525}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

...

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580793371}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580793682}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580793873}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580794014}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580794221}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580794441}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580838814}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580839382}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580840149}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580840960}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580841839}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580842782}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580868129}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580868721}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580869436}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580870110}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580870870}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580871560}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580931001}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580931867}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580932622}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580933617}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580934531}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580935434}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580965470}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580965831}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580966233}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580966684}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580967046}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1580967476}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

In [5]:
from datetime import datetime as dt
data = pd.read_parquet("dataframes/2020_comments/ElizabethWarren-2020_comments.csv")

# data["created_utc"] = data["created_utc"].apply(dt.fromtimestamp)
# data.to_csv('dataframes/2020_comments/2020_comments.csv', mode='a', header=False)
data


,created_utc,body,subreddit,score,author


In [7]:
## FUNCTIONS ##
def listToString(l):
    return ','.join(str(x) for x in l)

def getPushshiftData(sub, after = '2020-01-01', before = '2020-01-15', frequency='d'):
    """
    Get number of comments broken down daily from a specific subreddit between any time frame. 
    """
    url = 'https://api.pushshift.io/reddit/search/comment/?size=0&aggs=subreddit,created_utc&frequency=1'+ str(frequency) + '&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #url = 'https://api.pushshift.io/reddit/search/comment/?size=0&aggs=subreddit,created_utc&frequency=1d&after='+str(after)+'&before='+str(before)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['aggs']

def fixFailure(subreddit, after, before, frequency='d'):
    # break large subreddit into chunks of size maximum 500
    chunks = [subreddit[i:i + 500] for i in range(0, len(subreddit), 500)]
    sub = listToString(chunks[0])
    data = getPushshiftData(sub, after=after, before=before, frequency=frequency)
    
    # aggregate the values together
    for i in range(1, len(chunks)):
        time.sleep(2)
        sub = listToString(chunks[i])
        new_data = getPushshiftData(sub, after=after, before = before, frequency=frequency)
        
        for day in range(len(new_data['created_utc'])):
            data['created_utc'][day]['doc_count'] += new_data['created_utc'][day]['doc_count']
    
    return data

def pullData(name, subreddit, frequency='d', start = '2019-12-26', end = ''):
    #sub = listToString(subreddit)
    sub=subreddit
    try:
        data = getPushshiftData(sub, after = start, before = end, frequency=frequency)
    except:
        print("failed to retrieve group: " + name + "\n subreddit string: " + sub + "at index \n")
        print(str(i))
        #data = fixFailure(subreddit, after=start, before=end, frequency=frequency)
        data = None
    return data

def getdata(names, subreddits, start, end, frequency='d'):
    data = []
    for i, sub in enumerate(subreddits):
        data.append(pullData(names[i], sub, frequency=frequency, start=start, end=end))
        time.sleep(0.2)
    return data





def createDateColumn(df, col):
    df['date'] = df[col].apply(lambda x: datetime.datetime.fromtimestamp(x))
    return df

def createDataFrames(data):
    dfs = [pd.DataFrame(data[i]['created_utc']) for i in range(len(data))]
    for i in range(len(dfs)):
        dfs[i] = createDateColumn(dfs[i], 'key')
        dfs[i]['average_comments'] = dfs[i].doc_count.rolling(7, min_periods=7).mean()
    return dfs







def findAverageChange(df1, df2, offset = pd.Timestamp('2020-01-01')):
#     norm = df1.iloc[188:].doc_count.median()
    norm = normFunction(df1.append(df2, ignore_index=True), offset)
#     changes = df2[df2.date >= offset].doc_count/norm
    changes = changeFunction(df2, norm, offset)
    # find the median rate of change and return it
    return changes.median()

####### Change function ###################
#((dfs_20[0].doc_count - 6390)/6390*100).median()
### changefunction for changepoint and all dates after that #####
# def changeFunction(df, value, offset):
#     changes = (df[df.date >= offset].doc_count - value)/value * 100
#     return changes

### changefunction for changepoint and 2 weeks following it ###
def changeFunction(df, value, offset):
    temp = df[df.date >= offset]
    changes = (temp[temp.date <= offset + pd.DateOffset(weeks=2)].doc_count - value)/value * 100
    return changes

##### norm function for using previous 6 months as baseline ######
# def normFunction(df, offset):
#     temp = df[df.date < offset]
#     return temp[temp.date >= offset - pd.DateOffset(months=6)].doc_count.median()

### norm function for using january 2019 as baseline ####
def normFunction(df, offset):
    return df[df.date <= pd.Timestamp('2020-01-31')].doc_count.median()

# the default date is the mobility changepoint of the united states
def clusterAverages(df19, df20, names, mobility = pd.Timestamp('2020-03-16')):
    df = pd.DataFrame(names, columns=['name'])
    if isinstance(mobility, list):
        changes = [findAverageChange(df19[i], df20[i], mobility[i]) for i in range(len(df19))]
    else:
        changes = [findAverageChange(df19[i], df20[i], mobility) for i in range(len(df19))]
    df['changes'] = changes
    return df

In [9]:
left_candidates = ["JoeBiden","SandersForPresident","BaemyKlobaechar","ElizabethWarren","Pete_Buttigieg","YangForPresidentHQ"]

comments = getdata(left_candidates,left_candidates, start = '2019-12-26', end='')

https://api.pushshift.io/reddit/search/comment/?size=0&aggs=subreddit,created_utc&frequency=1d&after=2019-12-26&before=&subreddit=JoeBiden
https://api.pushshift.io/reddit/search/comment/?size=0&aggs=subreddit,created_utc&frequency=1d&after=2019-12-26&before=&subreddit=SandersForPresident
https://api.pushshift.io/reddit/search/comment/?size=0&aggs=subreddit,created_utc&frequency=1d&after=2019-12-26&before=&subreddit=BaemyKlobaechar
https://api.pushshift.io/reddit/search/comment/?size=0&aggs=subreddit,created_utc&frequency=1d&after=2019-12-26&before=&subreddit=ElizabethWarren
https://api.pushshift.io/reddit/search/comment/?size=0&aggs=subreddit,created_utc&frequency=1d&after=2019-12-26&before=&subreddit=Pete_Buttigieg
https://api.pushshift.io/reddit/search/comment/?size=0&aggs=subreddit,created_utc&frequency=1d&after=2019-12-26&before=&subreddit=YangForPresidentHQ


In [10]:
comments

[{'created_utc': [{'doc_count': 95, 'key': 1577318400},
   {'doc_count': 23, 'key': 1577404800},
   {'doc_count': 67, 'key': 1577491200},
   {'doc_count': 50, 'key': 1577577600},
   {'doc_count': 61, 'key': 1577664000},
   {'doc_count': 134, 'key': 1577750400},
   {'doc_count': 67, 'key': 1577836800},
   {'doc_count': 106, 'key': 1577923200},
   {'doc_count': 109, 'key': 1578009600},
   {'doc_count': 98, 'key': 1578096000},
   {'doc_count': 70, 'key': 1578182400},
   {'doc_count': 66, 'key': 1578268800},
   {'doc_count': 112, 'key': 1578355200},
   {'doc_count': 74, 'key': 1578441600},
   {'doc_count': 126, 'key': 1578528000},
   {'doc_count': 120, 'key': 1578614400},
   {'doc_count': 146, 'key': 1578700800},
   {'doc_count': 91, 'key': 1578787200},
   {'doc_count': 130, 'key': 1578873600},
   {'doc_count': 110, 'key': 1578960000},
   {'doc_count': 188, 'key': 1579046400},
   {'doc_count': 134, 'key': 1579132800},
   {'doc_count': 107, 'key': 1579219200},
   {'doc_count': 90, 'key': 15